In [1]:
# firebase_config.py
import firebase_admin
from firebase_admin import credentials, firestore

# Initialize Firebase app
cred = credentials.Certificate("serviceAccountKey.json")
firebase_admin.initialize_app(cred)

# Get Firestore client
db = firestore.client()

In [7]:

from google.cloud.firestore import SERVER_TIMESTAMP  # use this

def upsert_category(category_id, category_name, is_active=True, is_deleted=False):
    """
    Add a new category or update an existing one.
    """
    try:
        category_ref = db.collection("categories").document(category_id)
        doc = category_ref.get()

        if doc.exists:
            # Update existing category
            category_ref.update({
                "category_name": category_name,
                "updated_at": SERVER_TIMESTAMP,   # <-- use SERVER_TIMESTAMP directly
                "is_active": is_active,
                "is_deleted": is_deleted
            })
            print(f"Category {category_id} updated successfully!")
        else:
            # Create new category
            category_ref.set({
                "category_id": category_id,
                "category_name": category_name,
                "created_at": SERVER_TIMESTAMP,   # <-- use SERVER_TIMESTAMP directly
                "updated_at": SERVER_TIMESTAMP,   # <-- use SERVER_TIMESTAMP directly
                "is_active": is_active,
                "is_deleted": is_deleted
            })
            print(f"Category {category_id} added successfully!")

    except Exception as e:
        print("Error:", e)

def get_category(category_id):
    """
    Get a category by ID.
    """
    category_ref = db.collection("categories").document(category_id)
    doc = category_ref.get()
    if doc.exists:
        return doc.to_dict()
    else:
        print(f"Category {category_id} not found.")
        return None

def delete_category(category_id):
    """
    Delete a category.
    """
    category_ref = db.collection("categories").document(category_id)
    category_ref.delete()
    print(f"Category {category_id} deleted successfully!")

def list_categories():
    """
    List all categories.
    """
    categories = db.collection("categories").stream()
    for category in categories:
        print(category.id, category.to_dict())


In [11]:
# Add or update category
upsert_category("F101", "Furniture")

Category F101 added successfully!


In [13]:
# Get category
print(get_category("F201"))

{'is_deleted': False, 'created_at': DatetimeWithNanoseconds(2026, 1, 13, 8, 31, 3, 365000, tzinfo=datetime.timezone.utc), 'is_active': True, 'category_id': 'F201', 'category_name': 'Fashion', 'updated_at': DatetimeWithNanoseconds(2026, 1, 13, 8, 31, 3, 365000, tzinfo=datetime.timezone.utc)}


In [12]:
# List all categories
list_categories()

E101 {'is_deleted': False, 'created_at': DatetimeWithNanoseconds(2026, 1, 13, 8, 31, 43, 315000, tzinfo=datetime.timezone.utc), 'is_active': True, 'category_id': 'E101', 'category_name': 'Electronics', 'updated_at': DatetimeWithNanoseconds(2026, 1, 13, 8, 31, 43, 315000, tzinfo=datetime.timezone.utc)}
F101 {'is_deleted': False, 'created_at': DatetimeWithNanoseconds(2026, 1, 13, 8, 32, 23, 656000, tzinfo=datetime.timezone.utc), 'is_active': True, 'category_id': 'F101', 'category_name': 'Furniture', 'updated_at': DatetimeWithNanoseconds(2026, 1, 13, 8, 32, 23, 656000, tzinfo=datetime.timezone.utc)}
F201 {'is_deleted': False, 'created_at': DatetimeWithNanoseconds(2026, 1, 13, 8, 31, 3, 365000, tzinfo=datetime.timezone.utc), 'is_active': True, 'category_id': 'F201', 'category_name': 'Fashion', 'updated_at': DatetimeWithNanoseconds(2026, 1, 13, 8, 31, 3, 365000, tzinfo=datetime.timezone.utc)}
